<a href="https://colab.research.google.com/github/Orienfish/robust-lora/blob/emily_refactoring/GLNetSetup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Upload GLNet code from Github to Google Drive and navigate to folder. Should modify requirements.txt file before running to: 

numpy

torch

torchvision

tqdm

tensorboardX

Pillow==6.2.2

opencv-python



In [1]:
from __future__ import absolute_import, division, print_function
from google.colab import drive

drive.mount('/content/drive')
%cd drive/MyDrive/GLNet-master
%pip install -r requirements.txt


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GLNet-master


In [2]:
import os 
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from tqdm import tqdm
from dataset.deep_globe import DeepGlobe, classToRGB, is_image_file
from utils.loss import CrossEntropyLoss2d, SoftCrossEntropyLoss2d, FocalLoss
from utils.lovasz_losses import lovasz_softmax
from utils.lr_scheduler import LR_Scheduler

from helper import create_model_load_weights, get_optimizer, Trainer, Evaluator, collate, collate_test
from option import Options


from PIL import Image, ImageDraw

In [ ]:
torch.backends.cudnn.deterministic = True

n_class = 7

data_path = './data'
if not os.path.isdir(data_path): os.mkdir(data_path)

# added the creation of test and Sat folder to prevent future errors
test_path = './data/test'
if not os.path.isdir(test_path): os.mkdir(test_path)

test_Sat_path = './data/test/Sat'
if not os.path.isdir(test_Sat_path): os.mkdir(test_Sat_path)

model_path = './saved_models'
if not os.path.isdir(model_path): os.mkdir(model_path)

log_path = './runs'
if not os.path.isdir(log_path): os.mkdir(log_path)

task_name = 'test'

print(task_name)

###################################

# 1: train global; 2: train local from global; 3: train global from local
mode = 3 
evaluation = False
test = not evaluation    
print("mode:", mode, "evaluation:", evaluation, "test:", test)

batch_size = 1

###################################


if_crop = True
crop_image_name = 'test.jpg'
sub_image_dim = 1600


test
mode: 3 evaluation: False test: True


Upload image 'la.jpg' into ./data/test

Download pretrained models to saved_models folder.

In [ ]:
print("preparing datasets and dataloaders......")


def pad_image(pil_img, new_width, new_height, color):
    width, height = pil_img.size
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (0, 0))
    return result

def crop_image(pil_img, h_num, v_num, dim):  
    for i in range(v_num):
        for j in range(h_num):
            box = (j*dim, i*dim, j*dim+dim, i*dim+dim)
            sub_image = pil_img.crop(box).save(os.path.join(data_path, "test/Sat/" + 'crop_img_{}_{}.jpg'.format(i,j)), quality=95)
            
            
####### Image Cropping #########
if if_crop:
    original_image = Image.open(os.path.join(data_path, "test/" + crop_image_name))

    h_num = int(original_image.size[0]/sub_image_dim+0.5)
    v_num = int(original_image.size[1]/sub_image_dim+0.5)

    new_width = h_num*sub_image_dim
    new_height = v_num*sub_image_dim
    new_pad_iamge = pad_image(original_image, new_width, new_height, (0, 0, 0))
    new_pad_iamge.save(os.path.join(data_path, "test/" + 'pad_image.jpg'), quality=95)

    crop_image(new_pad_iamge, h_num, v_num, dim=sub_image_dim)


image_name_test = [image_name for image_name in os.listdir(os.path.join(data_path, "test", "Sat")) if is_image_file(image_name)]
print(image_name_test)

preparing datasets and dataloaders......
['crop_img_0_0.jpg', 'crop_img_0_1.jpg', 'crop_img_0_2.jpg', 'crop_img_1_0.jpg', 'crop_img_1_1.jpg', 'crop_img_1_2.jpg']


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset_test = DeepGlobe(os.path.join(data_path, "test"), image_name_test, label=False)
dataloader_test = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, num_workers=10, collate_fn=collate_test, shuffle=False, pin_memory=True)

###################################

size_g = 508
size_p = 508

##### sizes are (w, h) ##############################

# make sure margin / 32 is over 1.5 AND size_g is divisible by 4
size_g = (size_g, size_g) # resized global image
size_p = (size_p, size_p) # cropped local patch size
sub_batch_size = 6 # batch size for train local patches

path_g  = "fpn_deepglobe_global.pth"
path_g2l = "fpn_deepglobe_global2local.pth" 
path_l2g = "fpn_deepglobe_local2global.pth"

###################################

print("creating models......")

path_g = os.path.join(model_path, path_g)
path_g2l = os.path.join(model_path, path_g2l)
path_l2g = os.path.join(model_path, path_l2g)
model, global_fixed = create_model_load_weights(n_class, mode, evaluation, path_g=path_g, path_g2l=path_g2l, path_l2g=path_l2g)

##################################

evaluator = Evaluator(n_class, size_g, size_p, sub_batch_size, mode, test)

creating models......


In [ ]:
best_pred = 0.0
print("start testing......")

with torch.no_grad():
    model.eval()
    print("testing...")

    img_count = 0
    if if_crop:
        entire_seg_map = np.zeros([v_num*sub_image_dim, h_num*sub_image_dim], dtype=np.int8)
        
    if test: tbar = tqdm(dataloader_test)

    for i_batch, sample_batched in enumerate(tbar):
        # print("\nTesting {}/{} image \n".format(img_count+1, len(image_name_test)) )
        predictions, predictions_global, predictions_local = evaluator.eval_test(sample_batched, model, global_fixed)
        score_val, score_val_global, score_val_local = evaluator.get_scores()

        if mode == 1: tbar.set_description('global mIoU: %.3f' % (np.mean(np.nan_to_num(score_val_global["iou"])[1:])))
        else: tbar.set_description('agg mIoU: %.3f' % (np.mean(np.nan_to_num(score_val["iou"])[1:])))
        images = sample_batched['image']

        # print(images)

        if test:
            if not os.path.isdir("./prediction/"): os.mkdir("./prediction/")

            for i in range(len(images)):
                if mode == 1:
                    transforms.functional.to_pil_image(classToRGB(predictions_global[i]) * 255.).save("./prediction/" + 'sat_test_{}_mask.png'.format(img_count))
                    img_count = img_count+1
                    pred_seg_map = predictions_global[0]
                else:
                    transforms.functional.to_pil_image(classToRGB(predictions[i]) * 255.).save("./prediction/" + 'sat_test_{}_mask.png'.format(img_count))
                    img_count = img_count+1
                    pred_seg_map = predictions[0]
      

            if if_crop:
                left_index = ((img_count-1) % h_num)*sub_image_dim
                up_index = ((img_count-1)//h_num)*sub_image_dim
                entire_seg_map[up_index:up_index+sub_image_dim, left_index:left_index+sub_image_dim ] = pred_seg_map

seg_map_save_path = "./prediction/" + 'sat_test_{}_mask.png'.format('entire')
transforms.functional.to_pil_image(classToRGB(entire_seg_map) * 255.).save(seg_map_save_path)

  0%|          | 0/6 [00:00<?, ?it/s]

start testing......
testing...


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/content/drive/My Drive/GLNet-master/utils/metrics.py:42: RuntimeWarning: invalid value encountered in true_divide
  acc = np.nan_to_num(np.diag(hist) / hist.sum(axis=1))
/content/drive/My Drive/GLNet-master/utils/metrics.py:47: RuntimeWarning: invalid value encountered in true_divide
  iou = intersect / union
/content/drive/My Drive/GLNet-master/utils/metrics.py:50: RuntimeWarning: invalid value encountered in true_divide
  freq = hist.sum(axis=1) / hist.sum() # freq of each target
agg mIoU: 0.000: 100%|██████████| 6/6 [02:15<00:00, 22.62s/it]


In [12]:
from PIL import Image 
import numpy as np

# open image in working directory
image = Image.open("./prediction/sat_test_entire_mask.png")
orgimage = Image.open("./data/test/test.jpg")
width, height = orgimage.size

#use original image dimensions to crop new image
im1 = image.crop((0, 0, width, height)) 
im1.save('cropped_result', 'PNG')

# convert image into np array and save as npy file
data = np.array(im1)
print(data)
np.save('entire_seg_map.npy', data)

[[[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  ...
  [  0 255 255]
  [  0 255 255]
  [  0 255 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  ...
  [  0 255 255]
  [  0 255 255]
  [  0 255 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  ...
  [  0 255 255]
  [  0 255 255]
  [  0 255 255]]

 ...

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  ...
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  ...
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  ...
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]]
